In [2]:
#پروژه که برای خودم زدم روی انس جهانی که با دیپ لرنینگ زدم (RNN)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error

# بارگیری داده
data = pd.read_csv('D:\\python.list\\xau.csv')

# پردازش داده
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# تقسیم داده به داده‌های ورودی و خروجی
def create_dataset(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 100
X, y = create_dataset(scaled_data, time_step)

# تقسیم داده به داده‌های آموزش و آزمون
train_size = int(len(scaled_data) * 0.7)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# تغییر شکل داده برای ورود به مدل
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# ایجاد مدل LSTM
model = keras.Sequential()
model.add(keras.layers.LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(units=100, return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(units=100, return_sequences=False))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(units=1))

# کامپایل مدل
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error',metrics = ['mean_squared_error'])

# آموزش مدل
history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), verbose=1)

# پیش‌بینی قیمت طلا فردا
last_100_days = scaled_data[-100:]
last_100_days = last_100_days.reshape(1, last_100_days.shape[0], 1)
predicted_price = model.predict(last_100_days)
predicted_price = scaler.inverse_transform(predicted_price)
print("Gold price forecast:", predicted_price[0][0])

# محاسبه ضریب خطا mean squared error
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Score: {mse}%')


Epoch 1/200
69/69 [==============================] - 20s 216ms/step - loss: 0.0267 - mean_squared_error: 0.0267 - val_loss: 6.5856e-04 - val_mean_squared_error: 6.5856e-04
Epoch 2/200
69/69 [==============================] - 13s 191ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 5.5308e-04 - val_mean_squared_error: 5.5308e-04
Epoch 3/200
69/69 [==============================] - 11s 156ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 5.0415e-04 - val_mean_squared_error: 5.0415e-04
Epoch 4/200
69/69 [==============================] - 11s 159ms/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 5.9698e-04 - val_mean_squared_error: 5.9698e-04
Epoch 5/200
69/69 [==============================] - 10s 146ms/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 7.2682e-04 - val_mean_squared_error: 7.2682e-04
Epoch 6/200
69/69 [==============================] - 11s 156ms/step - loss: 0.0010 - mean_squared_error: 0.0010 - val_loss: 7.6566e-04 - val